In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

C:\Users\swdtk\AppData\Local\Temp\ipykernel_11864\2721483381.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def getRateDataFrame(codeStr):
    """
    関数の説明：
        codeStrで指定したレート一覧を、ヤフーファイナンスから取得する関数
    
    引数：
        codeStr:ヤフーファイナンスから取得する通貨名を指定

    戻値：
        rateDf:OHCLが記載されたDataFrame
    """    
    #ヤフーファイナンスからレートを取得
    rateDf = yf.download(codeStr, start='2004-01-01', end='2040-12-31')

    #不要な列の削除
    del rateDf['Adj Close']
    del rateDf['Volume']

    #indexを列に変換
    rateDf.reset_index(inplace= True)
    
    #parquetとして保存
    fileName =  './csv/' + codeStr + '.csv'
    rateDf.to_csv(fileName, encoding='cp932', index=False)

    return rateDf


def readRateDataFrame(codeStr):
    """
    関数の説明：
        getRateDataFrameでダウンロードしたCSVファイルを読み取る関数
    
    引数：
        codeStr:ヤフーファイナンスから取得する通貨名を指定

    戻値：
        rateDf:OHCLが記載されたDataFrame
    """
    codeValue = getRateCodeDic[codeStr]
    fileName =  './csv/' + codeValue + '.csv'
    rateDf = pd.read_csv(fileName, encoding='cp932', parse_dates=[0])

    return rateDf
    


def makeHendoRitu(rateDf, nRaw):
    """
    関数の説明：
        n行前の始値と、現在の始値の変動率を記載する
    
    引数：
        rateDf:Date、Open、High、Low、Closeで構成されるレートの一覧
        nRaw:どの期間の変動率を取得するか（1なら1行前）

    戻値：
        rateDf:変動率を記載したDataFrame
    """
    #n行前の始値と、現在の始値の変動率を記載
    nDaysStr = str(nRaw) + 'period'
    rateDf[nDaysStr] = rateDf['Open'].shift(nRaw)
    rateDf['変動率'] = (( rateDf['Open'] / rateDf[nDaysStr] ) -1 ) * 100
    del rateDf[nDaysStr]

    return rateDf

def makePeriodDataFrame(nRaw, readFlg=0):
    """
    関数の説明：
        各通貨の指定した期間の変動率を１つのDataFrameにまとめる関数
    
    引数：
        nRaw:指定した期間
        readFlg:
            0:保存済みのCSVファイルを読み込む
            1:新規にヤフーファイナンスからダウンロードする

    戻値：
        merge_Df:各通貨の指定した期間の変動率をまとめたDataFrame
    """
    merge_Df = pd.DataFrame()

    for codeKey in currencyList:
        #レート取得
        codeValue = getRateCodeDic[codeKey]
        if readFlg == 0:
            rateDf = readRateDataFrame(codeStr=codeKey)
        elif readFlg == 1: 
            rateDf = getRateDataFrame(codeStr=codeValue)

        #各通貨の指定した期間のDataFrameを取得する
        rateDf = makeHendoRitu(rateDf, nRaw=nRaw)
        rateDf = rateDf[['Date', '変動率']] #必要な列のみにする
        rateDf = rateDf.rename(columns={'変動率':codeKey}) #列名を通貨名に変更

        #取得してきた各通貨の変動率をマージしていく
        if len(merge_Df) == 0:
            merge_Df = rateDf
        else:
            merge_Df = pd.merge(merge_Df, rateDf, how='left', on='Date')
    
    fileName = str(nRaw) + '_hendoritsu.csv'
    merge_Df.to_csv('./csv/' + fileName, encoding='cp932', index=False)
    
    return merge_Df


def mergeZenjitsuhi(readFlg=0):
    """
    関数の説明：
        各通貨の前日比を計算してマージする
    
    引数：
        readFlg:
            0:保存済みのCSVファイルを読み込む
            1:新規にヤフーファイナンスからダウンロードする

    戻値：
        mergeZenjitsu_Df:マージしたDataFrame
    """
    mergeZenjitsu_Df = pd.DataFrame()

    for codeKey in currencyList:
        #レート取得
        codeValue = getRateCodeDic[codeKey]
        if readFlg == 0:
            rateDf = readRateDataFrame(codeStr=codeKey)
        elif readFlg == 1: 
            rateDf = getRateDataFrame(codeStr=codeValue)

        #各通貨の指定した期間のDataFrameを取得する
        rateDf['翌日の始値'] = rateDf['Open'].shift(-1)
        rateDf['翌日の始値比'] = ((rateDf['翌日の始値'] / rateDf['Open']) - 1) *100
        rateDf = rateDf[['Date', '翌日の始値比']] #必要な列のみにする
        rateDf = rateDf.rename(columns={'翌日の始値比':codeKey}) #列名を通貨名に変更

        #取得してきた各通貨の変動率をマージしていく
        if len(mergeZenjitsu_Df) == 0:
            mergeZenjitsu_Df = rateDf
        else:
            mergeZenjitsu_Df = pd.merge(mergeZenjitsu_Df, rateDf, how='left', on='Date')
    
    return mergeZenjitsu_Df


def getMaxMin(df):
    """
    関数の説明：
        makePeriodDataFrameで作成したDataFrameの最大列名と最小列名を取得する
    
    引数：
        df:makePeriodDataFrameで作成したDataFrame

    戻値：
        df:最大列名と最小列名を取得したDataFrame
    """
    # 最大値を持つ列を特定
    df['MAX列'] = df.apply(lambda row: row[currencyList].idxmax(), axis=1)

    # 最小値を持つ列を特定
    df['MIN列'] = df.apply(lambda row: row[currencyList].idxmin(), axis=1)
    
    return df

def mergeMaxMinResult(x,y,z):
    """
    関数の説明：
        getMaxMinで作成した3つの変動率を作成し、それをマージする関数
    
    引数：
        x=1つ目の期間
        y=2つ目の期間
        z=3つ目の期間

    戻値：
        merge_Df:3つの結果をマージしたDataFrame
    """
    merge_Df = pd.DataFrame()

    #指定した期間の変動率をまとめる
    part1Df = makePeriodDataFrame(nRaw=x, readFlg=0)
    #どの通貨が最大と最小かを求める
    part1Df = getMaxMin(df=part1Df)
    part1Df = part1Df[['Date','MAX列','MIN列']]

    #指定した期間の変動率をまとめる
    part2Df = makePeriodDataFrame(nRaw=y, readFlg=0)
    #どの通貨が最大と最小かを求める
    part2Df = getMaxMin(df=part2Df)
    part2Df = part2Df[['Date','MAX列','MIN列']]

    #指定した期間の変動率をまとめる
    part3Df = makePeriodDataFrame(nRaw=z, readFlg=0)
    #どの通貨が最大と最小かを求める
    part3Df = getMaxMin(df=part3Df)
    part3Df = part3Df[['Date','MAX列','MIN列']]


    merge_Df = pd.merge(part1Df, part2Df, how='left', on='Date')
    merge_Df = pd.merge(merge_Df, part3Df, how='left', on='Date')    
    
    return merge_Df

def selectCondition(type):
    """
    関数の説明：
        買いシグナルと売りシグナルのロジックを選択する関数
    
    引数：
        type:
            1:3つの期間がすべて一致したらTrue
            2:2つの期間が一致したらTure

    戻値：
        conditionBuy:買いシグナル
        conditionSell:売りシグナル
    """
    if type == 1:
        conditionBuy = (merge_Df['MAX列_x']==merge_Df['MAX列_y']) & ( merge_Df['MAX列_x']==merge_Df['MAX列'])
        conditionSell = (merge_Df['MIN列_x']==merge_Df['MIN列_y']) & ( merge_Df['MIN列_x']==merge_Df['MIN列'])
    
    elif type == 2:
        conditionBuy = (merge_Df['MAX列_x']==merge_Df['MAX列_y'])
        conditionSell = (merge_Df['MIN列_x']==merge_Df['MIN列_y'])
    
    
    return conditionBuy, conditionSell

In [19]:
readFlg = 0
stX, stY, stZ = 1,5,25
resultMerge_Df = pd.DataFrame()


stMerge = str(stX) + '_' + str(stY) + '_' +  str(stZ)

#取得する通貨
currencyList = ['USD', 'EUR', 'GBP', 'NZD', 'AUD']

#取得してくるコード一覧（すべてクロス円）
getRateCodeDic = {'USD':'USDJPY=X','EUR':'EURJPY=X', 'GBP':'GBPJPY=X', 'NZD':'NZDJPY=X', 'AUD':'AUDJPY=X'}

#各期間の最大最小をまとめる
merge_Df = mergeMaxMinResult(x=stX, y=stY, z=stZ)

#売買ロジックを選択する
conditionBuy, conditionSell = selectCondition(type=1)

#それぞれの期間が一致したらフラグを立てる
merge_Df['BuyFlg']=0
merge_Df.loc[conditionBuy, 'BuyFlg'] = merge_Df['MAX列_x']

merge_Df['SellFlg']=0
merge_Df.loc[conditionSell, 'SellFlg'] = merge_Df['MIN列_x']

#各通貨の現在のOpneと翌日のOpneとの比較結果をまとめる
mergeZenjitsu_Df = mergeZenjitsuhi(readFlg=readFlg)

#各通貨の変動率をマージ
hendoritsuDf = pd.read_csv('./csv/1_hendoritsu.csv', encoding='cp932', parse_dates=[0])
hendoritsuDic = {'USD':'USD変動率','EUR':'EUR変動率', 'GBP':'GBP変動率', 'NZD':'NZD変動率', 'AUD':'AUD変動率'}
hendoritsuDf = hendoritsuDf.rename(columns=hendoritsuDic)
merge_Df = pd.merge(merge_Df, hendoritsuDf, on='Date', how='left')

#merge_DfとmergeZenjitsu_Dfをマージ
result_df = pd.merge(merge_Df, mergeZenjitsu_Df, on='Date', how='left')

#結果をまとめる
result_df['結果_Buy'] = 0
result_df['結果_Sell'] = 0
for codeKey in currencyList:
    condition = (result_df['BuyFlg']==codeKey)
    result_df.loc[condition, '結果_Buy'] = result_df[codeKey]
    
    condition = (result_df['SellFlg']==codeKey)
    result_df.loc[condition, '結果_Sell'] = result_df[codeKey] * -1

#集計結果の合計値(買い)
gpBuyResultdf = result_df.groupby(['BuyFlg'])['結果_Buy'].sum().reset_index()
gpBuyResultdf = gpBuyResultdf[gpBuyResultdf['結果_Buy']!=0]

#集計結果の合計値(売り)
gpSellResultdf = result_df.groupby(['SellFlg'])['結果_Sell'].sum().reset_index()
gpSellResultdf = gpSellResultdf[gpSellResultdf['結果_Sell']!=0]

#合計値のDataFrameをマージ
resultMerge_Df = pd.merge(gpBuyResultdf, gpSellResultdf, how='left', left_on='BuyFlg', right_on='SellFlg')
resultMerge_Df['シミュレーションステータス']=stMerge

#CSV保存
result_df.to_csv(stMerge+'_result_df.csv', encoding='cp932', index=False)
resultMerge_Df.to_csv(stMerge+'_resultMerge_Df.csv', encoding='cp932', index=False)

C:\Users\swdtk\AppData\Local\Temp\ipykernel_11864\1367759412.py:159: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df['MAX列'] = df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_11864\1367759412.py:162: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df['MIN列'] = df.apply(lambda row: row[currencyList].idxmin(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_11864\1367759412.py:159: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df['MAX列'] = df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_11864\1367759412.py:159: FutureWarning: The behavior of Series.idxma

In [16]:

gpBuyResultdf

,BuyFlg,結果_Buy,シミュレーションステータス
1,AUD,-3.046448,1_5_25
2,EUR,-11.116889,1_5_25
3,GBP,11.813102,1_5_25
4,NZD,4.380896,1_5_25
5,USD,-29.684745,1_5_25


In [15]:
gpSellResultdf

,SellFlg,結果_Sell
1,AUD,-4.639755
2,EUR,0.414292
3,GBP,8.836822
4,NZD,-1.424590
5,USD,-0.129383


In [18]:

resultMerge_Df

,BuyFlg,結果_Buy,シミュレーションステータス,SellFlg,結果_Sell
0,AUD,-3.046448,1_5_25,AUD,-4.639755
1,EUR,-11.116889,1_5_25,EUR,0.414292
2,GBP,11.813102,1_5_25,GBP,8.836822
3,NZD,4.380896,1_5_25,NZD,-1.424590
4,USD,-29.684745,1_5_25,USD,-0.129383


In [17]:
df

,Date,Open,High,Low,Close
0,2004-01-01,80.292397,80.292397,80.292397,80.292397
1,2004-01-02,80.292397,81.209999,79.699997,81.089996
2,2004-01-05,81.059998,82.039497,81.040001,81.489998
3,2004-01-06,81.519997,82.032997,81.190002,81.550003
4,2004-01-07,81.639999,81.739998,81.010002,81.529999
...,...,...,...,...,...
5238,2024-02-13,97.542000,97.593002,97.169998,97.542999
5239,2024-02-14,97.264000,97.692001,97.071999,97.264000
5240,2024-02-15,97.825996,97.843002,97.325996,97.825996
5241,2024-02-16,97.843002,98.200996,97.704002,97.843002


C:\Users\swdtk\AppData\Local\Temp\ipykernel_9640\2771253278.py:2: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  part1Df['MAX列'] = part1Df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_9640\2771253278.py:2: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  part1Df['MAX列'] = part1Df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_9640\2771253278.py:2: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  part1Df['MAX列'] = part1Df.apply(lambda row: row[currencyList].idxmax(), axis=1)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_9640\2771253278.py:2: FutureWarning: The behavi

In [11]:
part1Df

,Date,USD,EUR,GBP,NZD,AUD,MAX,MAX列
0,2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5217,2024-02-09,1.039781,0.391339,0.344688,1.024122,-0.260187,1.039781,USD
5218,2024-02-12,0.693269,0.280785,0.219057,1.603473,-0.120077,1.603473,NZD
5219,2024-02-13,1.313503,0.732996,0.623588,1.198386,0.012305,1.313503,USD
5220,2024-02-14,2.276595,1.006999,1.416519,1.039295,0.086437,2.276595,USD


In [33]:
def makeSma(dataDf, tankiSma, choukiSma):
    """
    関数の説明：
        yfinanceで取得してきた株価のDataFrameを基に、
        ゴールデンクロスなら買い、デッドクロスなら決済のフラグを作成してくれる関数。
    
    Args:
        dataDf: yfinanceで取得してきた株価のDataFrame
        tankiSma: 短期移動平均線の日数
        choukiSma: 長期移動平均線の日数

    Returns:
        tradesDf: 売買フラグを付与したDataFrameを返す。
    """
    # 短期移動平均線を計算
    dataDf['短期SMA'] = dataDf['Adj Close'].rolling(window=tankiSma).mean()

    # 長期移動平均線を計算
    dataDf['長期SMA'] = dataDf['Adj Close'].rolling(window=choukiSma).mean()

    # ポジションを保持するための変数
    position = 0

    #ポジションをとった時のPrice
    numPositon = 0    

    # 取引履歴を保存するDataFrame
    tradesDf = pd.DataFrame(columns=['Date', 'Position', 'Price'])

    # 移動平均線を用いたバックテストを実施
    for i in range(len(dataDf)):
        # 短期移動平均線が長期移動平均線を上抜けした場合、買いポジションを取る
        if dataDf['短期SMA'][i] > dataDf['長期SMA'][i] and position == 0:
            position = 1
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)

        # 短期移動平均線が長期移動平均線を下抜けした場合、決済
        elif dataDf['短期SMA'][i] < dataDf['長期SMA'][i] and position == 1:
            position = 0
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
    
    #決済金額を追記
    tradesDf['result']=0

    for i in range(len(tradesDf)):
        #Buyなら買う
        if tradesDf.iloc[i, 1] == 'Buy':
            numPositon = tradesDf.iloc[i, 2]
        #Sellなら決済
        if tradesDf.iloc[i, 1] == 'Sell':
            tradesDf.iloc[i, 3] = (tradesDf.iloc[i, 2] - numPositon) / numPositon

    return tradesDf



In [6]:
n225_sma = makeSma(dataDf=n225_train, tankiSma=5, choukiSma=25)

C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\App

          Date Position         Price    result
0   2000-02-08      Buy  19868.880859  0.000000
1   2000-03-13     Sell  19189.929688 -0.034172
2   2000-03-24      Buy  19958.080078  0.000000
3   2000-04-18     Sell  18969.519531 -0.049532
4   2000-06-08      Buy  17004.339844  0.000000
..         ...      ...           ...       ...
235 2018-10-12     Sell  22694.660156 -0.005550
236 2018-11-12      Buy  22269.880859  0.000000
237 2018-11-15     Sell  21803.619141 -0.020937
238 2018-11-29      Buy  22262.599609  0.000000
239 2018-12-07     Sell  21678.679688 -0.026229

[240 rows x 4 columns]


In [7]:
n225_sma['result'].sum()

0.5362064095993865

In [12]:
columns = ['短期SMA', '長期SMA', '結果']
resultDf = pd.DataFrame(columns=columns)

In [35]:
resultDf.to_csv('result3.csv', index=False, encoding='cp932')



In [40]:
import warnings
warnings.filterwarnings('ignore')

#短期線
for i in range(51, 250):

    #長期線    
    for j in range(i+1, 300):
        
        n225_sma = makeSma(dataDf=n225_train, tankiSma=i, choukiSma=j)

        # 結果を保存
        new_row = {'短期SMA': i, '長期SMA': j, '結果': n225_sma['result'].sum()}
        resultDf = resultDf.append(new_row, ignore_index=True)
        
resultDf.to_csv('result.csv', index=False, encoding='cp932')

In [36]:
# 日経225のデータをyfinanceから取得
n225_test = yf.download('^N225', start='2019-01-01', end='2023-12-31')

# テストデータで検証
n225_sma = makeSma(dataDf=n225_test, tankiSma=195, choukiSma=296)

[*********************100%***********************]  1 of 1 completed


In [37]:
n225_sma['result'].sum()

0.2498771142944017

In [26]:
n225_sma.to_csv('test.csv')